## Market Basket Analysis Introduction

Companion notebook to http://pbpython.com/market-basket-analysis.html

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
df = pd.read_excel('../data/Online_Retail.xlsx')

In [ ]:
df.head()

In [ ]:
# Clean up spaces in description and remove any rows that don't have a valid invoice
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

In [ ]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [ ]:
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [ ]:
basket.head()

In [ ]:
# Show a subset of columns
basket.iloc[:,[0,1,2,3,4,5,6, 7]].head()

In [ ]:
# Convert the units to 1 hot encoded values
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1    

In [ ]:
basket_sets = basket.applymap(encode_units)

In [ ]:
# No need to track postage
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [ ]:
basket_sets.head()

In [ ]:
# Build up the frequent items
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [ ]:
frequent_itemsets.head()

In [ ]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

In [ ]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

In [ ]:
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

In [ ]:
basket['ALARM CLOCK BAKELIKE RED'].sum()

In [ ]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [ ]:
basket_sets2 = basket2.applymap(encode_units)

In [ ]:
basket_sets2.drop('POSTAGE', inplace=True, axis=1)

In [ ]:
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)

In [ ]:
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)
rules2

In [ ]:
rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5) ]